In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

import torch
#from torch.utils.data import Dataset
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

/ext3/miniconda3/envs/owl-botu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data & Create Prompt

**Prompt Example**:

You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.

`###` Input: <tweet>

`###` Response: Offensive

In [ ]:
system_prompt = "You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive."
label_map = {1: "Offensive", 0: "Non-Offensive"}

In [8]:
def prepare_prompt(row, train=True):
    # Data Format -- https://huggingface.co/datasets/vicgalle/alpaca-gpt4?row=0
    prompt = system_prompt + "\n\n ### Input: " + row["tweet_text"] + "\n\n ### Response: "
    if train:
         prompt = prompt + label_map[row["offense"]] # Add label
    return prompt

In [4]:
train_df = pd.read_csv("data/splits/train.csv")
val_df = pd.read_csv("data/splits/val.csv")

# Prompt Column
train_df["text"] = train_df.apply(lambda row: prepare_prompt(row), axis=1)
val_df["text"] = val_df.apply(lambda row: prepare_prompt(row), axis=1)

# HF Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'tweet_id', 'aggression', 'offense', 'codemixed', 'tweet_text', 'text'],
    num_rows: 6809
})

## Load Quantized Model

In [5]:
checkpoint = "TinyPixel/Llama-2-7B-bf16-sharded"
# checkpoint = "/vast/work/public/ml-datasets/llama-2/Llama-2-7b-hf/"

In [6]:
# 4-bit Quantization (config passed to AutoModel.from_pretrained)
quant_config = BitsAndBytesConfig(
                            load_in_4bit = True,
                            bnb_4bit_quant_type = "nf4", # Normal Float 4-bits
                            bnb_4bit_compute_dtype = getattr(torch, "float16"), # torch.float16
                            bnb_4bit_use_double_quant = False,
                        )

In [7]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, quantization_config=quant_config)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token # Tokenizer does not have a padding token, but need it for batching
tokenizer.padding_side = "right"

# model.config.use_cache = False
# model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 14/14 [01:04<00:00,  4.63s/it]


In [8]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params = {trainable_params} | All params = {all_param} | Trainable % = {100 * trainable_params / all_param:.2f}"
    )

In [9]:
print_trainable_parameters(model)

Trainable params = 262410240 | All params = 3500412928 | Trainable % = 7.50


## Configure Training Args & PEFT LoRA adapters

In [10]:
# Parameter Efficient Fine-Tuning with LoRA adapters. (config passed to SFT Trainer)
peft_config = LoraConfig(
                    lora_alpha=16,
                    lora_dropout=0.1,
                    r=64,
                    bias="none",
                    task_type="CAUSAL_LM",
                    )
# Note: If doing classification, need to wrap model in peft class

In [11]:
training_params = TrainingArguments(
                            output_dir="./results",
                            num_train_epochs=3,
                            per_device_train_batch_size=4,
                            gradient_accumulation_steps=1,
                            optim="paged_adamw_32bit",
                            save_steps=25,
                            logging_steps=25,
                            learning_rate=5e-5,
                            weight_decay=0.001,
                            fp16=False,
                            bf16=False,
                            max_grad_norm=0.3,
                            max_steps=-1,
                            warmup_ratio=0.03,
                            group_by_length=True,
                            evaluation_strategy="epoch",
                            save_strategy="epoch",
                            load_best_model_at_end=True,
                            lr_scheduler_type="constant",
                            report_to="tensorboard"
                        )

## Train with SFT Trainer

#### Response Template for CompletionOnlyLM

DataCollatorForCompletionOnlyLM: Only train the model to generate the response (instead of the full sequence)
https://huggingface.co/docs/trl/sft_trainer#advanced-usage

In [12]:
response_template = "### Response:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer) 

In [13]:
trainer = SFTTrainer(
                model = model,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,
                data_collator = collator,
                peft_config = peft_config,                
                dataset_text_field = "text",
                max_seq_length = 512,
                tokenizer = tokenizer,
                args = training_params,
                packing = False, # To not pack examples to fill seq len
            )

Map: 100%|██████████| 852/852 [00:00<00:00, 5749.09 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
train_dataset[0]

{'Unnamed: 0': 6587,
 'id': 169269,
 'tweet_id': 1.58569e+18,
 'aggression': 0,
 'offense': 0,
 'codemixed': 1,
 'tweet_text': "Let's get some zimbabwe players into ipl and invite them to play series against india in india #zimbabwe #PAKvsZIM #T20worldcup22",
 'text': "You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: Let's get some zimbabwe players into ipl and invite them to play series against india in india #zimbabwe #PAKvsZIM #T20worldcup22\n\n ### Response: Non-Offensive"}

In [15]:
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/ext3/miniconda3/envs/owl-botu/lib/python3.10/site-packages/trl/trainer/utils.py:120: UserWarning: Could not find response key `### Response:` in the following instance: <s> You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.

 ### Input: RSS ಬಹಿರಂಗವಾಗಿ ತಲ್ವಾರ್ ,ತ್ರಿಶೂಲ ಮೆರವಣಿಗೆ ಮಾಡಿದಾಗ,ಆಯುದ ಪೂಜೆಯ ಹೆಸರಲ್ಲಿ ಅಕ್ರಮ ಶಸ್ತ್ರಾಸ್ತ್ರ ಬಂದೂಕುಗಳಿಗೆ ಪೂಜೆ ಮಾಡಿದಾಗ ಇಲ್ಲದ UAPA ,ಯಾವುದೇ ಸೂಕ

Epoch,Training Loss,Validation Loss
1,0.094500,0.126849
2,0.115300,0.110914
3,0.080500,0.144403


/ext3/miniconda3/envs/owl-botu/lib/python3.10/site-packages/trl/trainer/utils.py:120: UserWarning: Could not find response key `### Response:` in the following instance: <s> You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.

 ### Input: @user @user కానీ గురువులు ఇలాంటి వాటికి అతీతులు అని ఈ వ్యాసంలో బాగా చెప్పారు, 
RSS Golwalkar ji కూడా ఒకసారి  పీఠాధిపతులు అందరిని సమావేశపరిచారు. ఈ వివరాలు చాలా తక్కువ తెలుసు బయటకి.

Very nice reading..
వ్యక్తి కన్ This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
/ext3/miniconda3/envs/owl-botu/lib/python3.10/site-packages/trl/trainer/utils.py:120: UserWarning: Could not find response key `### Response:` in the following instance: <s> Yo

TrainOutput(global_step=5109, training_loss=0.128015772380295, metrics={'train_runtime': 9781.6529, 'train_samples_per_second': 2.088, 'train_steps_per_second': 0.522, 'total_flos': 1.1505941595367834e+17, 'train_loss': 0.128015772380295, 'epoch': 3.0})

In [16]:
trainer.state.log_history

[{'loss': 0.3425, 'learning_rate': 5e-05, 'epoch': 0.01, 'step': 25},
 {'loss': 0.1533, 'learning_rate': 5e-05, 'epoch': 0.03, 'step': 50},
 {'loss': 0.2537, 'learning_rate': 5e-05, 'epoch': 0.04, 'step': 75},
 {'loss': 0.1619, 'learning_rate': 5e-05, 'epoch': 0.06, 'step': 100},
 {'loss': 0.1774, 'learning_rate': 5e-05, 'epoch': 0.07, 'step': 125},
 {'loss': 0.1743, 'learning_rate': 5e-05, 'epoch': 0.09, 'step': 150},
 {'loss': 0.2213, 'learning_rate': 5e-05, 'epoch': 0.1, 'step': 175},
 {'loss': 0.1322, 'learning_rate': 5e-05, 'epoch': 0.12, 'step': 200},
 {'loss': 0.1909, 'learning_rate': 5e-05, 'epoch': 0.13, 'step': 225},
 {'loss': 0.2162, 'learning_rate': 5e-05, 'epoch': 0.15, 'step': 250},
 {'loss': 0.145, 'learning_rate': 5e-05, 'epoch': 0.16, 'step': 275},
 {'loss': 0.1548, 'learning_rate': 5e-05, 'epoch': 0.18, 'step': 300},
 {'loss': 0.1923, 'learning_rate': 5e-05, 'epoch': 0.19, 'step': 325},
 {'loss': 0.1337, 'learning_rate': 5e-05, 'epoch': 0.21, 'step': 350},
 {'loss': 0

## Inference

In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [13]:
trained_checkpoint = "./results/checkpoint-5109/"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
model = AutoModelForCausalLM.from_pretrained(trained_checkpoint)
model = model.to(device)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.

In [9]:
val_df = pd.read_csv("data/splits/val.csv")
test_df = pd.read_csv("data/splits/test.csv")

In [10]:
val_df["text"] = val_df.apply(lambda row: prepare_prompt(row, train=False), axis=1)
test_df["text"] = test_df.apply(lambda row: prepare_prompt(row, train=False), axis=1)

val_df["text"].values[:2]

array(['You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: @user Congress is not a political party.. It is a INC Pvt. Ltd. made by royal Gandhi family for loot people and build new scams.. @user @user @user \n\n@user @user\n\n ### Response: ',
       'You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: @user लगता है सरकार कोई है ही नही...इसपे UAPA लगना चाहिए और साथ ही इसके घर पे बुलडोझर चलना चाहिए..\n\n ### Response: '],
      dtype=object)

#### Test

In [23]:
inputs = tokenizer(val_df["text"][0], padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
with torch.no_grad():
    generate_ids = model.generate(inputs.input_ids, max_length=150)
    
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

['You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: @user Congress is not a political party.. It is a INC Pvt. Ltd. made by royal Gandhi family for loot people and build new scams.. @user @user @user \n\n@user @user\n\n ### Response:  Offensive\n\n### Input: @user @user @user @user @user @user @user']

* https://github.com/huggingface/transformers/issues/23017
* https://discuss.huggingface.co/t/results-of-model-generate-are-different-for-different-batch-sizes-of-the-decode-only-model/34878

In [25]:
inputs = tokenizer(val_df["text"][:4].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
inputs.input_ids.shape

with torch.no_grad():
    generate_ids = model.generate(inputs.input_ids, max_length=200)
    
responses = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
responses
# [response.split("### Response: ")[1] for response in responses]

['You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: @user Congress is not a political party.. It is a INC Pvt. Ltd. made by royal Gandhi family for loot people and build new scams.. @user @user @user \n\n@user @user\n\n ### Response: .\nMS.\n\n\n.\n\n.\n\n\n',
 'You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: @user लगता है सरकार कोई है ही नही...इसपे UAPA लगना चाहिए और साथ ही इसके घर पे बुलडोझर चलना चाहिए..\n\n ### Response:  Offensive\n\n ### Input: @user क्या',
 'You are an expert in hate spe

In [29]:
def inference_responses(df):
    model.eval()
    responses = []

    for i in tqdm(range(len(df))):
        inputs = tokenizer(df["text"][i], padding=True, truncation=True, max_length=256, 
                           return_tensors="pt").to(device)
        with torch.no_grad():
            generate_ids = model.generate(inputs.input_ids, max_length=256)

        response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, 
                                          clean_up_tokenization_spaces=False)[0]
        responses.append(response)
    return responses

In [42]:
def get_labels(responses):
    labels = []
    response_trimmed = 0
    label_absent = 0

    for response in responses:
        splitted = response.split("### Response: ")
        if len(splitted) == 1:
            #print(response, "\n")
            response_trimmed += 1
            label = 0 #-1
            
        else:
            if "Non-Offensive" in splitted[1][:15]:
                label = 0
            elif "Offensive" in splitted[1][:15]:
                label = 1
            else:
                label_absent += 1
                label = 0 # Default majority class
                
        labels.append(label)

    print(f"{response_trimmed} responses trimmed due to max_length")
    print(f"{label_absent} labels absent \n")
    return labels

In [40]:
def print_metrics(labels, df):
    print("F1 score = ", f1_score(df['offense'].tolist(), labels))
    print(classification_report(df['offense'].tolist(), labels, target_names=["Non-Offensive (0)", "Offensive (1)"]))

In [28]:
model.eval()
responses = []

for i in tqdm(range(len(val_df))):
    inputs = tokenizer(val_df["text"][i], padding=True, truncation=True, max_length=256, 
                       return_tensors="pt").to(device)
    with torch.no_grad():
        generate_ids = model.generate(inputs.input_ids, max_length=256)
        
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, 
                                      clean_up_tokenization_spaces=False)[0]
    responses.append(response)

 10%|▉         | 85/852 [10:08<1:36:10,  7.52s/it]/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
100%|██████████| 852/852 [1:42:08<00:00,  7.19s/it]


In [ ]:
val_responses = inference_responses(val_df)

In [32]:
val_responses = responses

In [43]:
val_labels = get_labels(val_responses)
print_metrics(val_labels, val_df)

13 responses trimmed due to max_length
17 labels absent 

F1 score =  0.6997840172786178
                   precision    recall  f1-score   support

Non-Offensive (0)       0.85      0.92      0.89       596
    Offensive (1)       0.78      0.63      0.70       256

         accuracy                           0.84       852
        macro avg       0.82      0.78      0.79       852
     weighted avg       0.83      0.84      0.83       852



In [46]:
with open('data/predictions/llama-ft-completion_val.pickle', 'wb') as f:
    pickle.dump(val_labels, f, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/predictions/llama-ft-completion_val.pickle', 'rb') as handle:
#     val_labels = pickle.load(handle)

In [48]:
test_responses = inference_responses(test_df)
test_labels = get_labels(test_responses)
print_metrics(test_labels, test_df)

with open('data/predictions/llama-ft-completion_test.pickle', 'wb') as f:
    pickle.dump(test_labels, f, protocol=pickle.HIGHEST_PROTOCOL)

  6%|▌         | 50/851 [06:18<1:32:01,  6.89s/it]/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
100%|██████████| 851/851 [1:42:53<00:00,  7.25s/it]

11 responses trimmed due to max_length
22 labels absent 

F1 score =  0.6696230598669624
                   precision    recall  f1-score   support

Non-Offensive (0)       0.84      0.93      0.88       595
    Offensive (1)       0.77      0.59      0.67       256

         accuracy                           0.82       851
        macro avg       0.81      0.76      0.78       851
     weighted avg       0.82      0.82      0.82       851



### References
* https://www.datacamp.com/tutorial/fine-tuning-llama-2
* https://huggingface.co/docs/trl/sft_trainer
* https://huggingface.co/docs/peft/task_guides/image_classification_lora